In [28]:
#!pip install --quiet glvis
import numpy as np
import matplotlib.pyplot as plt
from  bfp import *
import mfem.ser as mfem
#from mfem.common.arg_parser import ArgParser
from glvis import glvis, GlvisData
import seaborn as sns
import pandas as pd
import os

In [29]:
class EDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, E_start, E_end):
        super(EDependentCoefficient, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class XDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, x_start, x_end):
        super(XDependentCoefficient, self).__init__()
        self.x_start = x_start
        self.x_end = x_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.x_bins = np.linspace(x_start, x_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value

        x_val = x[0]
        for i in range(self.n_groups - 1):
            if np.isclose(x_val, self.x_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if self.x_bins[i] <= x_val < self.x_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class MyVectorCoefficient(mfem.VectorConstantCoefficient):
    def __init__(self, mu_coeff, S_coeff):
        super(MyVectorCoefficient, self).__init__(2)
        self.mu_coeff = mu_coeff 
        self.S_coeff = S_coeff

    def EvalValue(self, x):
        mu_val = self.mu_coeff.EvalValue(x) 
        S_val = self.S_coeff.EvalValue(x) 
        return [mu_val, S_val]
    
class InflowCoefficientSN(mfem.PyCoefficient):
    def __init__(self, inflow, mu):
        super(InflowCoefficientSN, self).__init__()
        self.inflow = inflow
        self.mu = mu
    def EvalValue(self, x):
        if self.mu > 0:
            return self.inflow
        else:
            return self.inflow
        
class ConstantVector(mfem.VectorPyCoefficient):
    def __init__(self, constant):
        super(ConstantVector, self).__init__(1)
        self.constant = constant 
    def EvalValue(self, x):
        return self.constant
    

class QFuncCoefficient(mfem.PyCoefficient):
    def __init__(self, pn, a_val=0, b_val=0, xs_t_val=0, mu_val=0, S_val=0):
        super(QFuncCoefficient, self).__init__()
        self.pn = pn
        self.a_val = a_val
        self.b_val = b_val
        self.xs_t_val = xs_t_val
        self.mu_val = float(mu_val)
        self.S_val = S_val

    def EvalValue(self, x):
        if self.pn == 3:
            return self.coeff_pn3(x)
        elif self.pn == 4:
            return self.coeff_pn4(x)
        elif self.pn == 5:
            return self.coeff_pn5(x)

    def coeff_pn3(self, x):
        return float(self.mu_val * self.b_val + self.xs_t_val * (self.a_val + self.b_val * x[0]))

    def coeff_pn4(self, x):
        return float(self.xs_t_val * (self.a_val + self.b_val * x[1]) + self.S_val * self.b_val)

    def coeff_pn5(self, x):
        print("Debug: self.mu =", self.mu_val, "x =", x)
        sol =  self.mu_val * self.b_val * x[1] \
            + self.xs_t_val * (self.a_val + self.b_val * x[0] * x[1]) \
            + self.S_val * self.b_val * x[0]
        return sol
    

In [30]:
# Set the problem parameters
nx = 10
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0.0
E_end = 1.0
N_ang = 2
order = 1
inflow = 5.0
S_const = 2.0
xs_t_const = 5.0
q_const = 0.0
alpha = 1.0
beta = 0.5
a = 5.0
b = 10.0

# Create mesh
mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
mesh.UniformRefinement()
dim = mesh.Dimension()

# Define finite element space
fec = mfem.DG_FECollection(order, dim, btype=0)
fes = mfem.FiniteElementSpace(mesh, fec)

mu_vals, w_vals = gauss_legendre_dirs(N_ang)
#mu_vals, w_vals = [-0.577], [1.0]

# Define boundary attributes
ess_tdof_list1 = mfem.intArray()
dir_bdr1 = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr1.Assign(0)
dir_bdr1[3] = 0
dir_bdr1[1] = 0

ess_tdof_list2 = mfem.intArray()
dir_bdr2 = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr2.Assign(0)
dir_bdr2[0] = 1
dir_bdr2[2] = 1

# Set coefficients
S_coeff = StoppingPowerCoefficientE(S_const, E_start, E_end)
xs_t_coeff = TotalXSCoefficientE(xs_t_const, E_start, E_end)

# Solution
psi_mu = []
psi_mu_list = []

for i in range(len(mu_vals)):
    a = 5.0
    b = 10.0

    mu = mu_vals[i]
    w = w_vals[i]
    print("  Solving for mu =", mu)
    mu_coeff = ConstantCoefficient(mu)
    inflow_coeff = InflowCoefficient(inflow, mu)
    v_coeff1 = VectorConstCoefficient(mu, 0.0)
    v_coeff2 = VectorConstCoefficient(0.0, S_const)
    q_coeff = QFuncCoefficient(pn=4, a_val=a, b_val=b, xs_t_val=xs_t_const, mu_val=mu, S_val=S_const)
    

    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff1, 1.0))
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff2, 1.0))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)), dir_bdr1)
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)), dir_bdr2)
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff1, -alpha, -beta), dir_bdr1)
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff2, -alpha, -beta), dir_bdr2)
    b.Assemble()
    
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)

    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(1000)
    solver.SetKDim(30)
    solver.SetPrintLevel(0)
    solver.Mult(b, psi)
    

    g = glvis((mesh, psi), 500,500, keys='ARjmcbp')
    g.set_size(1000, 1000)
    g.render()
    
    psi.Save("psi_hs_mu_{:.3f}.gf".format(mu))
    psi_mu.append(psi.GetDataArray())
    psi_mu_list.append((mu, w, psi))

phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, w, X in psi_mu_list:
    phi.Add(w, X)

phi.Save("phi_dg.gf")




File '/Users/melekderman/github/BFP/melek/mesh/usr/10x10_2D.mesh' already exists.
  Solving for mu = -0.5773502691896257


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.5773502691896257


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [31]:
g = glvis((mesh, phi), 500,500, keys='ARjmcbp')
g.set_size(900, 900)
g.render()

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [32]:
g = glvis((mesh, phi), 500,500, keys='rljgac//0')
g.set_size(900, 900)
g.render()


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [12]:
def sol (x, E, xs_t, a, b, mu, S):
    return mu * b * x + xs_t * (a + b * x * E) + S * b * x

nx = 10
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0.0
E_end = 1.0
N_ang = 2
order = 1
inflow = 5.0
S_const = 2.0
xs_t_const = 5.0
q_const = 0.0
alpha = 1.0
beta = 0.5
a = 5
b = 10
x = np.linspace(x_start, x_end, nx)
E = np.linspace(E_start, E_end, nE)
mu=0.577
for i in x:
    for j in E:
        print(f'x = {i}, E = {j}, {sol(i, j, xs_t_const, a, b, mu, S_const)}')

x = 0.0, E = 0.0, 25.0
x = 0.0, E = 0.1111111111111111, 25.0
x = 0.0, E = 0.2222222222222222, 25.0
x = 0.0, E = 0.3333333333333333, 25.0
x = 0.0, E = 0.4444444444444444, 25.0
x = 0.0, E = 0.5555555555555556, 25.0
x = 0.0, E = 0.6666666666666666, 25.0
x = 0.0, E = 0.7777777777777777, 25.0
x = 0.0, E = 0.8888888888888888, 25.0
x = 0.0, E = 1.0, 25.0
x = 0.1111111111111111, E = 0.0, 27.863333333333333
x = 0.1111111111111111, E = 0.1111111111111111, 28.480617283950615
x = 0.1111111111111111, E = 0.2222222222222222, 29.097901234567903
x = 0.1111111111111111, E = 0.3333333333333333, 29.715185185185184
x = 0.1111111111111111, E = 0.4444444444444444, 30.332469135802473
x = 0.1111111111111111, E = 0.5555555555555556, 30.949753086419754
x = 0.1111111111111111, E = 0.6666666666666666, 31.567037037037036
x = 0.1111111111111111, E = 0.7777777777777777, 32.184320987654324
x = 0.1111111111111111, E = 0.8888888888888888, 32.80160493827161
x = 0.1111111111111111, E = 1.0, 33.41888888888889
x = 0.222222